# 3 Implémentation

In [2]:
import random as rd

In [3]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1: tuple, c2: tuple) -> None:
        """
        Ajout d'un mur entre les cellules c1 et c2
        :param c1: tuple (l,c) des coordonnées de la première cellule
        :param c2: tuple (l,c) des coordonnées de la seconde cellule
        :return: None
        """
        # On teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1: tuple, c2: tuple) -> None:
        """
        Suppression d'un mur entre les cellules c1 et c2
        :param c1: tuple (l,c) des coordonnées de la première cellule
        :param c2: tuple (l,c) des coordonnées de la seconde cellule
        :return: None
        """
        # On teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de la suppression d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Suppression du mur
        if c2 not in self.neighbors[c1]:  # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2)    # on l'ajoute
        if c1 not in self.neighbors[c2]:  # Si c3 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1)    # on l'ajoute
            
    def get_walls(self) -> list:
        """
        Renvoie la liste des murs du labyrinthe
        :return: list[list[tuple]] : liste des murs du labyrinthe
        """
        walls = []
        for i in range(self.height):
            for j in range(self.width):
                if (i+1, j) not in self.neighbors[(i,j)] and i+1 < self.height:
                    walls.append([(i,j), (i+1, j)])
                if (i, j+1) not in self.neighbors[(i,j)] and j+1 < self.width:
                    walls.append([(i,j), (i, j+1)])
        return walls
    
    def fill(self) -> None:
        """
        Remplissage complet du labyrinthe
        :return: None
        """
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in {(i, j) for i in range(self.height) for j in range(self.width)}:
                if c1 != c2:
                    self.add_wall(c1, c2)
                    
    def empty(self) -> None:
        """
        Vidage complet du labyrinthe
        :return: None
        """
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in {(i, j) for i in range(self.height) for j in range(self.width)}:
                if c1 != c2:
                    self.remove_wall(c1, c2)
                    
    def get_contiguous_cells(self, c: tuple):
        """
        Renvoie la liste des cellules contiguës à la cellule c
        :param c: tuple (l,c) des coordonnées de la cellule
        :return: list[tuple] : liste des cellules contiguës à c
        """
        contiguous = []
        
        if c[0] > 0:
            contiguous.append((c[0]-1, c[1]))
        if c[0] < self.height-1:
            contiguous.append((c[0]+1, c[1]))
        if c[1] > 0:
            contiguous.append((c[0], c[1]-1))
        if c[1] < self.width-1:
            contiguous.append((c[0], c[1]+1))
        
        return contiguous
        
    
    def get_reachable_cells(self, c: tuple):
        """
        Renvoie la liste des cellules accessibles depuis la cellule c
        :param c: tuple (l,c) des coordonnées de la cellule
        :return: list[tuple] : liste des cellules accessibles depuis c
        """
        reachable = []
        for cell in self.get_contiguous_cells(c):
            if cell in self.neighbors[c]:
                reachable.append(cell)
                
        return reachable
    
    @classmethod
    def gen_btree(cls, h: int, w: int) -> 'Maze':
        """
        Génération d'un labyrinthe de dimensions height x width
        selon la méthode de l'arbre binaire
        :param h: int : hauteur du labyrinthe
        :param w: int : largeur du labyrinthe
        :return: Maze : labyrinthe généré
        """
        laby = cls(h, w)
        laby.fill()
        
        for i in range(h):
            print('i', i)
            for j in range(w):
                print('j', j)
                cell = (i, j)
                if i < h-1 and j < w-1:
                    if (i+1, j) not in laby.neighbors[cell] and (i, j+1) not in laby.neighbors[cell]:
                        laby.remove_wall(cell, (i+1, j) if rd.choice([True, False]) else (i, j+1))
                elif i < h-1:
                    if (i+1, j) not in laby.neighbors[cell]:
                        laby.remove_wall(cell, (i+1, j))
                elif j < w-1:
                    if (i, j+1) not in laby.neighbors[cell]:
                        laby.remove_wall(cell, (i, j+1))
                        
        return laby
    
    @classmethod
    def gen_sidewinder(cls, h: int, w: int) -> 'Maze':
        """
        Génération d'un labyrinthe de dimensions height x width
        selon la méthode du sidewinder
        :param h: int : hauteur du labyrinthe
        :param w: int : largeur du labyrinthe
        :return: Maze : labyrinthe généré
        """
        laby = cls(h, w)
        laby.fill()
        
        for i in range(h - 1):
            print('i', i)
            sequence = []
            for j in range(w - 1):
                print('j', j)
                cell = (i, j)
                sequence.append(cell)
                if rd.choice([True, False]):
                    laby.remove_wall(cell, (i, j + 1))
                else:
                    random_cell = rd.choice(sequence)
                    laby.remove_wall(random_cell, (random_cell[0] + 1, random_cell[1]))
                    sequence = []
            sequence.append((i, w - 1))
            random_cell = rd.choice(sequence)
            laby.remove_wall(random_cell, (random_cell[0] + 1, random_cell[1]))
        
        for j in range(w - 1):
            cell = (h - 1, j)
            laby.remove_wall(cell, (h - 1, j + 1))
                    
        return laby
    
    @classmethod
    def gen_fusion(cls, h: int, w: int) -> 'Maze':
        """
        Génération d'un labyrinthe de dimensions height x width
        selon la méthode de la fusion de chemin
        :param h: int : hauteur du labyrinthe
        :param w: int : largeur du labyrinthe
        :return: Maze : labyrinthe généré
        """
        laby = cls(h, w)
        laby.fill()
        labels = {cell: i for i, cell in enumerate(laby.neighbors)}
        walls = laby.get_walls()
        rd.shuffle(walls)
        
        for wall in walls:
            c1, c2 = wall
            if labels[c1] != labels[c2]:
                laby.remove_wall(c1, c2)
                label1, label2 = labels[c1], labels[c2]
                for cell in labels:
                    if labels[cell] == label2:
                        labels[cell] = label1
                
        return laby
    
    @classmethod
    def gen_exploration(cls, h: int, w: int) -> 'Maze':
        """
        Génération d'un labyrinthe de dimensions height x width
        selon la méthode de l'exploration exhaustive
        :param h: int : hauteur du labyrinthe
        :param w: int : largeur du labyrinthe
        :return: Maze : labyrinthe généré
        """
        laby = cls(h, w)
        laby.fill()
        cell = (rd.randint(0, h-1), rd.randint(0, w-1))
        visited = [cell]
        stack = [cell]
        
        while len(stack) != 0:
            cell = stack.pop()
            contiguous = laby.get_contiguous_cells(cell)
            unvisited = [c for c in contiguous if c not in visited]
            if len(unvisited) != 0:
                stack.append(cell)
                next_cell = rd.choice(unvisited)
                laby.remove_wall(cell, next_cell)
                visited.append(next_cell)
                stack.append(next_cell)
                
        return laby
    
    @classmethod
    def gen_wilson(cls, h: int, w: int) -> 'Maze':
        """
        Génération d'un labyrinthe de dimensions height x width selon la méthode de Wilson
        :param h: int : hauteur du labyrinthe
        :param w: int : largeur du labyrinthe
        :return: Maze : labyrinthe généré
        """
        """
        Génération d'un labyrinthe de dimensions height x width
        selon la méthode de Wilson
        :param h: int : hauteur du labyrinthe
        :param w: int : largeur du labyrinthe
        :return: Maze : labyrinthe généré
        """
        laby = cls(h, w)
        laby.fill()
        unvisited = [(i, j) for i in range(h) for j in range(w)]
        cell = rd.choice(unvisited)
        unvisited.remove(cell)
        
        while len(unvisited) != 0:
            cell = rd.choice(unvisited)
            path = [cell]
            while cell in unvisited:
                cell = rd.choice(laby.get_contiguous_cells(cell))
                if cell in path:
                    path = path[:path.index(cell)+1]
                else:
                    path.append(cell)
            for i in range(len(path)-1):
                laby.remove_wall(path[i], path[i+1])
                unvisited.remove(path[i])
                
        return laby
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def solve_dfs(self, start, stop):
        """
        Résolution du labyrinthe par la méthode de la recherche en profondeur
        :param start: tuple : Cellule de départ
        :param stop: tuple : Cellule d'arrivée
        :return: list[tuple] : Chemin de la cellule de départ à la cellule d'arrivée
        """
        stack = [start]
        unvisited = [(i, j) for i in range(self.height) for j in range(self.width)]
        unvisited.remove(start)
        pred = {start: start}
        finish = False
        
        while len(unvisited) != 0 and not finish:
            c = stack.pop()
            if c == stop:
                finish = True
            else:
                for neighbor in self.get_reachable_cells(c):
                    if neighbor in unvisited:
                        unvisited.remove(neighbor)
                        stack.append(neighbor)
                        pred[neighbor] = c
                        
        path = []
        c = stop
        while c != start:
            path.append(c)
            c = pred[c]
        path.append(start)
        
        return path
    
    def solve_bfs(self, start, stop):
        """
        Résolution du labyrinthe par la méthode de la recherche en largeur
        :param start: tuple : Cellule de départ
        :param stop: tuple : Cellule d'arrivée
        :return: list[tuple] : Chemin de la cellule de départ à la cellule d'arrivée
        """
        queue = [start]
        unvisited = [(i, j) for i in range(self.height) for j in range(self.width)]
        unvisited.remove(start)
        pred = {start: start}
        finish = False
        
        while len(unvisited) != 0 and not finish:
            c = queue.pop(0)
            if c == stop:
                finish = True
            else:
                for neighbor in self.get_reachable_cells(c):
                    if neighbor in unvisited:
                        unvisited.remove(neighbor)
                        queue.append(neighbor)
                        pred[neighbor] = c
                        
        path = []
        c = stop
        while c != start:
            path.append(c)
            c = pred[c]
        path.append(start)
        
        return path
    
    def solve_rhr(self, start, stop):
        """
        Résolution du labyrinthe par la méthode de la main droite
        :param start: tuple : Cellule de départ
        :param stop: tuple : Cellule d'arrivée
        :return: list[tuple] : Chemin de la cellule de départ à la cellule d'arrivée
        """
        finish = False
        path = [start]
        
        direction = None
        neighbors = self.get_reachable_cells(start)
        if neighbors[0][0] == start[0] + 1:
            direction = 'S'
        elif neighbors[0][0] == start[0] - 1:
            direction = 'N'
        elif neighbors[0][1] == start[1] + 1:
            direction = 'E'
        elif neighbors[0][1] == start[1] - 1:
            direction = 'O'
        
        while not finish:
            c = path[-1]
            if c == stop:
                finish = True
            else:
                neighbors = self.get_reachable_cells(c)
                if direction == 'N':
                    if (c[0], c[1]+1) in neighbors:
                        path.append((c[0], c[1]+1))
                        direction = 'E'
                    elif (c[0]-1, c[1]) in neighbors:
                        path.append((c[0]-1, c[1]))
                    else:
                        direction = 'O'
                elif direction == 'S':
                    if (c[0], c[1]-1) in neighbors:
                        path.append((c[0], c[1]-1))
                        direction = 'O'
                    elif (c[0]+1, c[1]) in neighbors:
                        path.append((c[0]+1, c[1]))
                    else:
                        direction = 'E'
                elif direction == 'E':
                    if (c[0]+1, c[1]) in neighbors:
                        path.append((c[0]+1, c[1]))
                        direction = 'S'
                    elif (c[0], c[1]+1) in neighbors:
                        path.append((c[0], c[1]+1))
                    else:
                        direction = 'N'
                elif direction == 'O':
                    if (c[0]-1, c[1]) in neighbors:
                        path.append((c[0]-1, c[1]))
                        direction = 'N'
                    elif (c[0], c[1]-1) in neighbors:
                        path.append((c[0], c[1]-1))
                    else:
                        direction = 'S'
                             
        return path
    
    def distance_geo(self, c1, c2):
        """
        Calcul de la distance géodésique entre deux cellules
        :param c1: tuple : Coordonnées de la première cellule
        :param c2: tuple : Coordonnées de la seconde cellule
        :return: float : Distance géodésique entre c1 et c2
        """
        path = self.solve_bfs(c1, c2)
        return len(path) - 1
    
    def distance_man(self, c1, c2):
        """
        Calcul de la distance de Manhattan entre deux cellules
        :param c1: tuple : Coordonnées de la première cellule
        :param c2: tuple : Coordonnées de la seconde cellule
        :return: int : Distance de Manhattan entre c1 et c2
        """
        return abs(c1[0] - c2[0]) + abs(c1[1] - c2[1])
    
    def worst_path_len(self, start):
        """
        Calcul de la longueur du plus long chemin menant du départ à une impasse (une feuille)
        :param start: tuple : Coordonnées de la cellule de départ
        :return: int : Longueur du plus long chemin dans le labyrinthe
        """
        path_len = 0
        for cell in self.neighbors:
            if len(self.neighbors[cell]) == 1:
                path = self.solve_bfs(start, cell)
                if len(path) > path_len:
                    path_len = len(path)
        return path_len
    
    def dead_end_number(self):
        """
        Calcul du nombre de culs-de-sac dans le labyrinthe
        :return: int : Nombre de culs-de-sac dans le labyrinthe
        """
        dead_ends = 0
        for cell in self.neighbors:
            if len(self.neighbors[cell]) == 1:
                dead_ends += 1
        return dead_ends
    

In [4]:
laby = Maze(4, 4)
print(laby.info())

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente


In [5]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛


In [6]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛


In [7]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛


In [8]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛


In [9]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)


In [10]:
laby.neighbors[(2, 3)].remove((1,3))

In [11]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a un mur entre (1, 3) et (2, 3) car (1, 3) n'est pas dans le voisinage de (2, 3) et (2, 3) n'est pas dans le voisinage de (1, 3)


In [12]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"{c1} est accessible depuis {c2} et vice-versa")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"{c1} n'est pas accessible depuis {c2} et vice-versa")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

(1, 3) n'est pas accessible depuis (2, 3) et vice-versa


In [13]:
L = []
for i in range(laby.height):
    for j in range(laby.width):
        L.append((i,j))
print(f"Liste des cellules : \n{L}")

Liste des cellules : 
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


# 4 Manipulation de labyrinthes

In [14]:
laby = Maze(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛


In [15]:
laby = Maze(5, 5)
laby.empty()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛


In [16]:
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛


In [17]:
laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛


In [18]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛


In [19]:
print(laby.get_walls())

[[(0, 0), (0, 1)], [(0, 1), (1, 1)]]


In [20]:
print(laby.get_contiguous_cells((0,1)))

[(1, 1), (0, 0), (0, 2)]


In [21]:
print(laby.get_reachable_cells((0,1)))

[(0, 2)]


# 5 Génération

## 5.1 Arbre binaire

In [22]:
laby = Maze.gen_btree(4, 4)
print(laby)

i 0
j 0
j 1
j 2
j 3
i 1
j 0
j 1
j 2
j 3
i 2
j 0
j 1
j 2
j 3
i 3
j 0
j 1
j 2
j 3
┏━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ┫
┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ┫
┃   ┃           ┃
┣   ╋━━━╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛


In [23]:
laby = Maze.gen_sidewinder(4, 4)
print(laby)

i 0
j 0
j 1
j 2
i 1
j 0
j 1
j 2
i 2
j 0
j 1
j 2
┏━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋   ┫
┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛


In [24]:
laby = Maze.gen_fusion(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃           ┃       ┃       ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃       ┃   ┃   ┃   ┃   ┃   ┃           ┃           ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃               ┃       ┃   ┃       ┃           ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃               ┃   ┃       ┃                   ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃           ┃   ┃       ┃       ┃   ┃   ┃   ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃       ┃                   ┃       ┃       ┃       ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━┫
┃   ┃   ┃           ┃       ┃           ┃               ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃   ┃   ┃               ┃           ┃   ┃               ┃   ┃
┣   ╋   

In [25]:
laby = Maze.gen_exploration(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃                       ┃           ┃           ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃           ┃   ┃               ┃       ┃   ┃       ┃       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━┫
┃   ┃   ┃       ┃   ┃       ┃       ┃   ┃           ┃       ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃   ┃       ┃       ┃   ┃       ┃       ┃       ┃           ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃       ┃           ┃   ┃           ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ┫
┃       ┃   ┃       ┃   ┃   ┃       ┃           ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃   ┃       ┃           ┃   ┃       ┃   ┃           ┃       ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━┫
┃       ┃       ┃       ┃   ┃               ┃               ┃
┣   ╋━━━

In [26]:
laby = Maze.gen_wilson(12, 12)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃                   ┃   ┃       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃       ┃   ┃               ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━┫
┃   ┃   ┃       ┃               ┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃       ┃       ┃   ┃       ┃                   ┃
┣━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃       ┃       ┃       ┃   ┃   ┃       ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃           ┃           ┃           ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃       ┃   ┃       ┃       ┃   ┃       ┃       ┃
┣━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━┫
┃       ┃           ┃                   ┃       ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃       ┃               ┃       ┃       ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃   ┃           ┃   ┃   ┃           ┃


# 6 Résolution

## 6.1 Résolution par parcours

In [27]:
laby = Maze.gen_fusion(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D ┃                                       ┃       ┃   ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃ * ┃           ┃   ┃                   ┃           ┃       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃ * ┃   ┃ *   *   *   * ┃   ┃       ┃   ┃   ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃ *   *   * ┃   ┃   ┃ *   * ┃                   ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃   ┃   ┃       ┃         *   *   * ┃           ┃       ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━┫
┃                               ┃ *   *     ┃   ┃           ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━┫
┃       ┃   ┃           ┃   ┃   ┃   ┃ * ┃   ┃       ┃   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃               ┃           ┃       ┃ *             ┃   ┃   ┃
┣   ╋━━━

In [28]:
laby = Maze.gen_exploration(15, 15)
solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D         ┃           ┃       ┃           ┃               ┃
┣   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃ * ┃               ┃   ┃   ┃   ┃   ┃   ┃       ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━┫
┃ *   *   * ┃       ┃       ┃           ┃           ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃ *   *   * ┃           ┃       ┃       ┃   ┃           ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃ * ┃                       ┃           ┃   ┃   ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃ *   *   *   *   *   *   *   * ┃   ┃       ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃           ┃               ┃ * ┃       ┃   ┃               ┃
┣━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━┫
┃   ┃       ┃   ┃   ┃ *   *   * ┃   ┃       ┃   ┃           ┃
┣   ╋   

## 6.2 Résolution en aveugle : « la main droite »

In [29]:
laby = Maze.gen_exploration(15, 15)
solution = laby.solve_rhr((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D     ┃                                   ┃               ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃ * ┃           ┃   ┃ *   *   * ┃   ┃       ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━┫
┃ * ┃ *   * ┃   ┃ *   * ┃ * ┃ * ┃       ┃   ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃ *   * ┃ * ┃     * ┃ *   * ┃ * ┃   ┃   ┃                   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃ * ┃ *   * ┃ *   * ┃ * ┃ * ┃ * ┃       ┃           ┃   ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃ *   * ┃     * ┃ *   * ┃ *   * ┃   ┃       ┃   ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━┫
┃ *   *   *   * ┃ *   * ┃           ┃           ┃           ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃ * ┃ *   *   * ┃ *   * ┃   ┃   ┃   ┃           ┃ *   * ┃   ┃
┣   ╋   

# 7 Évaluation

In [30]:
laby = Maze.gen_exploration(5, 5)
print(laby)
print(laby.distance_geo((0, 0), (4, 4)))
print(laby.distance_man((0, 0), (4, 4)))
print(laby.worst_path_len((0, 0)))
print(laby.dead_end_number())

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋   ╋   ╋   ┫
┃           ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━┫
┃           ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ┫
┃   ┃       ┃       ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛

10
8
12
5
